# 词典合并去重

In [18]:
def rm_dupli(path):
    with open(path, 'r') as fp1:
        word_list = fp1.readlines()
        word_rmdu = set([word.strip() for word in word_list])
    save_path = path + '_rmdu'
    with open(save_path, 'w') as fp2:
        fp2.write('\n'.join(word_rmdu))
    print 'The num of', path[14:], 'is', len(word_rmdu)

In [19]:
neg_path = 'data/sen_dict/sen_neg_dict'
pos_path = 'data/sen_dict/sen_pos_dict'
neg_pos_path = 'data/sen_dict/sen_neg_pos_dict'
rm_dupli(neg_path)
rm_dupli(pos_path)
rm_dupli(neg_pos_path)

The num of sen_neg_dict is 13752
The num of sen_pos_dict is 10219
The num of sen_neg_pos_dict is 23633


In [148]:
# 生成停用词表
with open('data/sen_dict/sen_neg_pos_dict', 'r') as fp:
    word_list = fp.readlines()
    sen_words = set([word.strip().decode('utf-8') for word in word_list])
with open('data/stop_words_ch.txt', 'r') as fp:
    word_list = fp.readlines()
    stop_words = [word.strip().decode('gbk') for word in word_list]
new_stop_words = [w.encode('utf-8') for w in stop_words if w not in sen_words]
with open('data/new_stop_words_ch.txt', 'w') as fp:
    fp.write('\n'.join(new_stop_words))

# CHI

In [6]:
import os
import jieba.posseg
from tqdm import tqdm
from gensim import corpora

In [120]:
# 测试
processed_corpus = [['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]
# 创建词-词频字典
dictionary = corpora.Dictionary(processed_corpus)
# 查看词典
print dictionary.token2id
# 查看词在文档集中出现的文档数
print dictionary.dfs

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}
{0: 2, 1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 3, 7: 3, 8: 2, 9: 3, 10: 3, 11: 2}


In [2]:
# 分词、去除停用词、去重、词性过滤
def sent2word(sentence):
    segList = set(jieba.posseg.cut(sentence))
    newSent = [w.word for w in segList if w.word not in stop_words and w.flag in pos]
    return newSent

In [8]:
neg_path = 'data/ChnSentiCorp_htl_unba_10000/neg'
pos_path = 'data/ChnSentiCorp_htl_unba_10000/pos'
# 停用词表
with open('data/new_stop_words_ch.txt', 'r') as fp:
    word_list = fp.readlines()
    stop_words = set([word.strip() for word in word_list])
# 词性过滤表
pos = 'v vd vn vshi vyou vf vx vi vl vg a ad an ag al d'.split()
def corpus2list(path):
    corpus = []
    for parent, _, fileNames in os.walk(path):
        for fileName in tqdm(fileNames):
            currentPath = os.path.join(parent, fileName)
            with open(currentPath, 'r') as fp:
                text = fp.read().decode('gbk', 'ignore').strip()
                corpus.append(sent2word(text))
    return corpus

In [9]:
neg_corpus = corpus2list(neg_path)
pos_corpus = corpus2list(pos_path)

100%|██████████| 7000/7000 [01:53<00:00, 61.79it/s] 


In [14]:
neg_corpus_dict = corpora.Dictionary(neg_corpus)
pos_corpus_dict = corpora.Dictionary(pos_corpus)
del neg_corpus
del pos_corpus

In [45]:
# 计算CHI
n = 10000.0
neg_num = 3000.0
pos_num = 7000.0
dfs_neg = neg_corpus_dict.dfs  # 词id在文档集中出现的文档数
dfs_pos = pos_corpus_dict.dfs
neg_chi = {}
pos_chi = {}
for i, word in neg_corpus_dict.items():
    a = dfs_neg[i]
    try:
        b = dfs_pos[pos_corpus_dict.token2id[word]]
    except:
        b = 0
    c = neg_num - a
    d = pos_num - b
    x = n*(a*d-b*c)*(a*d-b*c)/neg_num/pos_num/(a+b)/(c+d)  # 计算卡方检验
    neg_chi[word] = x

for i, word in pos_corpus_dict.items():
    a = dfs_pos[i]
    try:
        b = dfs_neg[neg_corpus_dict.token2id[word]]
    except:
        b = 0
    c = pos_num - a
    d = neg_num - b
    x = n*(a*d-b*c)*(a*d-b*c)/neg_num/pos_num/(a+b)/(c+d)  # 计算卡方检验
    pos_chi[word] = x

In [46]:
sort_neg_chi = sorted(neg_chi.items(), key=lambda i:i[1], reverse=True)
sort_pos_chi = sorted(pos_chi.items(), key=lambda i:i[1], reverse=True)

In [63]:
print ' '.join([i[0] for i in sort_neg_chi[100:120]])
print ' '.join([i[0] for i in sort_pos_chi[100:120]])

预定 已经 甚至 喜欢 洗 周到 折腾 完全 不敢 睡觉 低 入住 开门 值得 从来 直到 打扫 可怜 敢 退
已经 甚至 喜欢 洗 周到 折腾 完全 不敢 睡觉 低 入住 开门 值得 从来 直到 打扫 可怜 敢 退 见


In [59]:
sort_neg_chi[1000:1010]

[(u'\u51e1', 7.002100630189057),
 (u'\u722c\u51fa', 7.002100630189057),
 (u'\u505a\u7231', 7.002100630189057),
 (u'\u6d53\u539a', 7.002100630189057),
 (u'\u76ee\u7779', 7.002100630189057),
 (u'\u8e66\u8fea', 7.002100630189057),
 (u'\u771f\u76f8', 7.002100630189057),
 (u'\u6781\u70ed', 7.002100630189057),
 (u'\u79df\u7ed9', 7.002100630189057),
 (u'\u6ca1\u64e6', 7.002100630189057)]

In [58]:
sort_pos_chi[1000:1010]

[(u'\u53cd\u8fc7\u6765', 5.7250960025759525),
 (u'\u8bf7\u793a', 5.7250960025759525),
 (u'\u6a21\u7cca', 5.7250960025759525),
 (u'\u61c2\u5f97', 5.7250960025759525),
 (u'\u5f97\u4e0d\u5230', 5.7250960025759525),
 (u'\u4e8f', 5.7250960025759525),
 (u'\u4e0d\u4fe1', 5.7250960025759525),
 (u'\u8bd5\u95ee', 5.7250960025759525),
 (u'\u4e92\u76f8', 5.7250960025759525),
 (u'\u7ad9', 5.663932569804846)]